# CHARISMA: Demonstration of Chada wavenumber calibration

In [ ]:
from ramanchada import chada,chada_utilities,chada_io


Create a CHADA archive from native Raman file

In [ ]:
import os,os.path
print(os.path.expanduser("~"))
chada_file = os.path.join(os.path.expanduser("~"),"Desktop","chada","200218-17.cha")
print(chada_file)

In [ ]:
#DATA_FILES = r"\\vm-stor-51\alle\Projekte Kunststoffe\RD\MA\Öffentliche_Projekte\2020-01-07_230007_H2020-CHARISMA_barton\Python\charisma-raman-spectrum-harmonization\test_data\\"
DATA_FILES = "D:\\nina\DATA\CHARISMA"
source_file = os.path.join(DATA_FILES,"200218-17.wdf")
print(source_file)
if not os.path.exists(source_file):
    print("{} does not exist!".format(source_file))

In [ ]:
chada_io.create(source_file, chada_file)
if not os.path.exists(chada_file):
    print("{} does not exist!".format(chada_file))

## 1. Generate calibration using a reference spectrum with equal wavenumber range

Load CHADA archive into Chada object

In [ ]:
C = chada.Chada(chada_file)

Show unprocessed data

In [ ]:
C.plot()

Now separate a baseline

In [ ]:
#C.baseline()
C.fit_baseline()
C.remove_baseline()

The data has been updated

In [ ]:
C.plot()

Crop spectrum k direction

In [ ]:
C.x_crop(500, 2500)

In [ ]:
C.plot()

Area normalization

In [ ]:
C.normalize('area')

commit saves the CHADA

In [ ]:
C.commit('unshifted')

In [ ]:
print(C.path,C.commits)


Now, a copy of the unshifted CHADA is made and saved as "C:\Users\barton\Desktop\chada\200218-17_shifted.cha"

> NJ - is it supposed to be manual copy outside the notebook ?  Added a cell to do so


In [ ]:
from shutil import copyfile

shifted_path = C.path.replace(".cha","_shifted.cha")
copyfile(C.path,shifted_path)
C.path = shifted_path

In [ ]:
_shift=5
print("Shift spectrum in x direction by a constant {} rel. 1/cm".format(_shift))

In [ ]:
import numpy as np
C.shiftX(np.ones_like(C.x_data)*_shift)

In [ ]:
C.plot()

In [ ]:
 
C.path
print("commit saves the shifted CHADA to {}".format(C.path))

In [ ]:
C.commit('shifted')


In [ ]:
print(C.path,C.commits)

A CHADA Group can be used to show the relative shift

In [ ]:

G = chada.ChadaGroup([chada_file,shifted_path])

In [ ]:
%matplotlib notebook

In [ ]:
G.plot()

In [ ]:
from ramanchada import chada_calibration

The shifts between both CHADA files is calculated

In [ ]:
peak_pos, shift_pos = chada_calibration.makeXCalFromSpec(shifted_path, chada_file, bounds=[-10.,10.])

~5 rel. 1/cm is the correct result !

In [ ]:
shift_pos

The function makeXCalFromSpec() correctly finds the top-hat shift function that was applied

The CHADA method .XCal() calibrates the current CHADA using the determined shift anchors (orange circles).

In [ ]:
C.XCal(peak_pos, shift_pos, show=True)

## 2. Generate calibration using a reference spectrum with different wavenumber range

The CHADA is reset to the raw state

In [ ]:
C.rewind(0)

In [ ]:
C.plot()

In [ ]:
print(C.baseline)
#C.baseline()
# C.baseline is an array, not a function and returns error 'numpy.ndarray' object is not callable
C.fit_baseline()
C.remove_baseline()
C.normalize()

In [ ]:
C.plot()

A baseline-corrected and normalized version is stored, this time with full wavenumber range.

In [ ]:
C.commit('rewind')

In [ ]:
from shutil import copyfile

target_path = chada_file.replace(".cha","_target.cha")
copyfile(C.path,target_path)

print(target_path)

In [ ]:
print("Now, a copy of the unshifted CHADA is made as '{}', and loaded from disk into CHADA Cs".format(target_path))

In [ ]:
Cs = chada.Chada(target_path)

In [ ]:
Cs.plot()

The target data is cropped

In [ ]:
Cs.x_crop(500, 1500)

In [ ]:
Cs.plot()

In [ ]:
Cs.shiftX(np.ones_like(Cs.x_data)*5)

commit saves the cropped CHADA.

In [ ]:
Cs.commit('shifted')

The shifts between both CHADA files is calculated

In [ ]:
peak_pos, shift_pos = chada_calibration.makeXCalFromSpec(target_path, chada_file, bounds=[-10.,10.])

In [ ]:
shift_pos

The function makeXCalFromSpec() correctly finds the top-hat shift function that was applied

The CHADA method .XCal() calibrates the current CHADA using the determined shift anchors (orange circles).

In [ ]:
Cs.XCal(peak_pos, shift_pos, show=True)

## 3. Generate calibration with non-constant wavenumber shift using a reference spectrum with different wavenumber range

This time, a ramp-shaped shift is applied from 0 to 5 rel. 1/cm

In [ ]:
Cs.shiftX(np.linspace(0, 5, len(Cs.x_data)))

In [ ]:
Cs.commit('shifted')

In [ ]:
Cs.plot()

The shifts between both CHADA files is calculated. Remember, "C:\Users\barton\Desktop\chada\200218-17.cha" already has a constant shift of 5 rel. 1/cm! 

In [ ]:
peak_pos, shift_pos = makeXCalFromSpec(target_file, chada_file, bounds=[-10.,10.])

In [ ]:
shift_pos

The relative shift function, a top hat plus ramp, is correctly determined and compensated by the .XCal() method.

In [ ]:
Cs.XCal(peak_pos, shift_pos, show=True)

In [ ]:
Cs.commit('calibrated with ramp shift')